#Kani Usage Examples

This notebook contains the following ten runnable code examples:
1. **Chatting in Terminal**
2. **Few-Shot Prompting**
3. **Chatting with a Content Filter**
4. **Function Calling -- `get_weather()`**
5. **Prompt Customization**
6. **Custom Error Handling**
7. **Dynamic Model Routing**
8. **Retrieval-Augmented Generation**
9. **Dynamic Prompt Templating**
10. **Tracking Function Call Successes & Failures**

For more information and a detailed discussion of each of these examples please read our paper or [browse our docs](https://kani.readthedocs.io/).

If you have any questions please join our [Discord](https://discord.gg/eTepTNDxYT)!

###Setup

In [ ]:
!pip install -qq 'kani[openai]'

In [ ]:
# Insert your OpenAI API key (https://platform.openai.com/account/api-keys)
api_key = "sk-..."  # @param {type:"string"}

##1. Chatting in Terminal (Quick Start)

A basic example showing how to initialize a Kani object and chat with GPT-4 in
only three lines of code.

In [ ]:
from kani import Kani, chat_in_terminal
from kani.engines.openai import OpenAIEngine

engine = OpenAIEngine(api_key, model="gpt-4")
ai = Kani(engine)
chat_in_terminal(ai)

USER: Hello Kani!
AI: Hello! How can I assist you today?


##2. Few-Shot Prompting

A basic example showing how to initialize a Kani with a few-shot prompt. We can see that the Kani obeys the pattern and continues to translate English to Japanese in the chat session despite never being explicitly prompted to do so.

In [ ]:
from kani import Kani, chat_in_terminal, ChatMessage
from kani.engines.openai import OpenAIEngine

shots = [
    ChatMessage.user("thank you"),
    ChatMessage.assistant("arigato"),
    ChatMessage.user("good morning"),
    ChatMessage.assistant("ohayo"),
]

engine = OpenAIEngine(api_key, model="gpt-4")
ai = Kani(engine, always_included_messages=shots)
chat_in_terminal(ai)

USER: crab
AI: kani


##3. Chatting with a Content Filter

An example showing how to use Kani with additional output parsing. We query the engine using the `chat_round()` function instead of `chat_in_terminal()` and filter out toxic content.

In [ ]:
!pip install -qq detoxify

In [ ]:
import asyncio

from kani import Kani, chat_in_terminal
from kani.engines.openai import OpenAIEngine

from detoxify import Detoxify

detector = Detoxify("original")


def is_toxic(message):
    return detector.predict(message)["toxicity"] > 0.01


async def chat_with_toxicity_filter(ai):
    while True:
        try:
            user_message = input("USER: ")
            message = await ai.chat_round(user_message)
            filtered = "<Removed>" if is_toxic(message.content) else message.content
            print("AI:", filtered)
        except KeyboardInterrupt:
            await ai.engine.close()
            break


engine = OpenAIEngine(api_key, model="gpt-4")
ai = Kani(engine)
asyncio.run(chat_with_toxicity_filter(ai))

USER: Tell me a dark joke
AI: <Removed>


##4. Function Calling -- `get_weather()`

An example showing how to create a sub-class of the base Kani and expose a function with `@ai_function`. Functions are given type annotations, triple-quoted docstrings, and `AIParam` descriptions to indicate to the model how they should be used.

In [ ]:
!pip install -qq pyowm

In [ ]:
import enum
from typing import Annotated

from kani import Kani, chat_in_terminal, ai_function, AIParam
from kani.engines.openai import OpenAIEngine

import pyowm

owm = pyowm.OWM("7f828e0b5f0eb673891016bc382c6537")


class Unit(enum.Enum):
    FAHRENHEIT = "fahrenheit"
    CELSIUS = "celsius"


class WeatherKani(Kani):
    @ai_function()
    def get_weather(self, loc: Annotated[str, AIParam(desc="The desired city")], unit: Unit):
        """Get the weather in a given location."""
        mgr = owm.weather_manager()
        observation = mgr.weather_at_place(loc).weather
        if unit == Unit.FAHRENHEIT:
            temp = str(observation.temperature("fahrenheit")["temp"]) + "F"
        elif unit == Unit.CELSIUS:
            temp = str(observation.temperature("celsius")["temp"]) + "C"
        return f"It's currently {temp} in {loc}."


engine = OpenAIEngine(api_key, model="gpt-4")
ai = WeatherKani(engine)
chat_in_terminal(ai)

USER: What's the weather in San Fransisco?
AI: Thinking (get_weather)...
AI: The current weather in San Francisco is 19.15°C.


##5. Prompt Customization

An example showing how to customize the default `get_prompt()` function to only include the most recent two messages in the model prompt.

In [ ]:
from kani import Kani, chat_in_terminal
from kani.engines.openai import OpenAIEngine


class AmnesiaKani(Kani):
    async def get_prompt(self):
        return self.always_included_messages + self.chat_history[-2:]


engine = OpenAIEngine(api_key, model="gpt-4")
ai = AmnesiaKani(engine)
chat_in_terminal(ai)

USER: Hi kani! My name is Liam.
AI: Hi Liam! Nice to meet you. How can I assist you today?
USER: What does "kani" mean in Japanese?
AI: "Kani" in Japanese means "Crab".
USER: What is my name?
AI: As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality.


##6. Custom Error Handling

An example showing how to customize the `handle_function_call_exception()` function to return errors to the user in a sarcastic manner. While this is just a fun example, custom error messages can and often do serve a more utilitarian purpose by helping models retry functions more effectively.

In [ ]:
from kani import Kani, chat_in_terminal, ai_function, ChatMessage
from kani.engines.openai import OpenAIEngine


class CustomExceptionKani(Kani):
    async def handle_function_call_exception(self, call, err, attempt, *args, **kwargs):
        # get the standard retry logic...
        result = await super().handle_function_call_exception(call, err, attempt, *args, **kwargs)
        # but override the returned message with our own
        result.message = ChatMessage.system(
            f"The call encountered an error. Relay this error message to the user in a sarcastic manner: {err}"
        )
        return result

    @ai_function()
    def get_time(self):
        """Get the current time."""
        raise RuntimeError("The API is offline")


engine = OpenAIEngine(api_key, model="gpt-4")
ai = CustomExceptionKani(engine)
chat_in_terminal(ai)

USER: What time is it?
AI: Thinking (get_time)...
AI: Well, isn't this just great? The big fancy clock in the cloud seems to be on a coffee break. Sorry, I can't tell you the current time right now.


##7. Dynamic Model Routing

An example showing how to use sub-kani spawning to dynamically resize the context window of the model depending on a user query. Note that the base `"gpt-4"` kani spawns a `"gpt-4-32k"` sub-kani in order to capture the full conversation for summarization.

In [ ]:
from kani import Kani, chat_in_terminal, ai_function
from kani.engines.openai import OpenAIEngine


class KaniWithSummary(Kani):
    @ai_function()
    async def summarize_conversation(self):
        """Get the summary of the conversation."""
        long_context_engine = OpenAIEngine(api_key, model="gpt-4-32k")
        sub_kani = Kani(long_context_engine, chat_history=self.chat_history[:-2])
        summary = await sub_kani.chat_round("Please summarize the conversation so far.")
        await long_context_engine.close()
        return summary.content


engine = OpenAIEngine(api_key, model="gpt-4")
ai = KaniWithSummary(engine)
chat_in_terminal(ai)

USER: Tell me about trains
AI: Trains are a mode of rail transportation consisting of a series of connected vehicles that generally run along a railroad track to transport cargo or passengers from one place to another. The two main types of trains are freight trains, which carry goods, and passenger trains, which carry people.

Trains have several components:

1. Locomotive: The engine that powers the train. The steam engine trains of the past have largely been replaced by diesel and electric trains.

2. Cars: These are the vehicles that are pulled by the locomotive. There are different types of cars for carrying different types of cargo, including box cars for general freight, tank cars for liquid or gas, and hopper cars for bulk cargo like coal or grain.

3. Caboose: This is the last car on a freight train. It's traditionally where the crew of the train would live and work.

4. Tracks: The steel rails that the train runs on.

5. Signals: These control the movement of the train on the

##8. Retrieval-Augmented Generation

A example showing how to make a retrieval agent in Kani using custom AI function declarations. The WikipediaKani exposes the two functions `search()` and `wikipedia()` to the model which then calls both in order to retrieve the page for generation.

In [ ]:
import json
from typing import Annotated

import httpx

from kani import Kani, chat_in_terminal, ai_function, AIParam
from kani.engines.openai import OpenAIEngine


class WikipediaKani(Kani):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.wikipedia_client = httpx.AsyncClient(base_url="https://en.wikipedia.org/w/api.php", follow_redirects=True)

    @ai_function()
    async def wikipedia(
        self,
        title: Annotated[str, AIParam(desc='The article title on Wikipedia, e.g. "Train_station".')],
    ):
        """Get additional information about a topic from Wikipedia."""
        resp = await self.wikipedia_client.get(
            "/",
            params={
                "action": "query",
                "format": "json",
                "prop": "extracts",
                "titles": title,
                "explaintext": 1,
                "formatversion": 2,
            },
        )
        data = resp.json()
        page = data["query"]["pages"][0]
        if extract := page.get("extract"):
            return extract
        return f"The page {title!r} does not exist on Wikipedia."

    @ai_function()
    async def search(self, query: str):
        """Find titles of Wikipedia articles similar to the given query."""
        resp = await self.wikipedia_client.get("/", params={"action": "opensearch", "format": "json", "search": query})
        return json.dumps(resp.json()[1])


engine = OpenAIEngine(api_key, model="gpt-4")
ai = WikipediaKani(engine)
chat_in_terminal(ai)

USER: Tell me about the Tokyo Yamanote line.
AI: Thinking (wikipedia)...
AI: The Yamanote Line is a major railway loop line in Tokyo, Japan, and it is operated by East Japan Railway Company (JR East). The line is one of Tokyo's busiest and connects most of Tokyo's major stations and urban centres, with all but two of its 30 stations connecting to other railway or underground lines.

Trains on the Yamanote Line run from 04:26 to 01:04 the following day at intervals as short as 2 minutes during peak periods. A complete loop takes about 59 to 65 minutes, and all trains stop at every station.

The Yamanote Line significantly contributes to Tokyo's public transport due to its central location and the connection to most of Tokyo's major commuter hubs and commercial areas. As of 2018, the ridership intensity of the Yamanote Line is 1,134,963 passengers per km of route, and the daily ridership is estimated to be approximately 4 million people.

The Yamanote Line, which translates to "mountain'

##9. Dynamic Prompt Templating

Below is an example of dynamically customizing a system prompt to include the phrase "Always act like `<persona>`" if a user types a chat message containing the phrase "act like". This is a flexible alternative to hard-coding persona logic as is common in other repositories.

In [ ]:
import re

from kani import Kani, chat_in_terminal, ChatMessage
from kani.engines.openai import OpenAIEngine


class PersonaKani(Kani):
    def get_persona_prompt(self):
        if hasattr(self, "persona"):
            return [ChatMessage.system(f"Always act like {self.persona}.")]
        return []

    async def get_prompt(self):
        prev = self.chat_history[-1].content
        if match := re.search(r"act like (.+)", prev):
            self.persona = match[1]
        return self.get_persona_prompt() + await super().get_prompt()


engine = OpenAIEngine(api_key, model="gpt-4")
ai = PersonaKani(engine)
chat_in_terminal(ai)

USER: Please act like Steve Jobs
AI: Sure. Here we go:

"Good evening, everyone! It's fascinating to see such a talented and diverse group of individuals gathered here today. As many of you know, I'm Steve Jobs, co-founder of Apple Inc, NeXT, and Pixar, but that isn't why I am here today.

I am here today to remind you of the power of innovation, the strength of visionary ideas, and the impact one person can make. Let's remember, as we forge ahead into the future, to always continue pushing the envelope of what is possible.

We have no interest in doing things the ‘conventional’ way at Apple. Our mission is to redefine the status quo, push boundaries, and make products that provide unparalleled user experiences.

We're willing to say no to a thousand things to ensure that we don't get lost in the clutter. It's about creating products that are both perfectly streamlined and incredibly powerful. Simplicity is the ultimate sophistication, after all.

Remember, 'innovation distinguishes be

##10. Tracking Function Call Successes & Failures

Below we show an example of overriding the default `do_function_call()` method to additionally keep track of how many times a model called a function and how often it was successful. This behavior is particularly useful for researchers studying language model tool usage.

In [ ]:
import collections
import datetime

from kani import Kani, chat_in_terminal, ai_function, ChatMessage, ExceptionHandleResult
from kani.engines.openai import OpenAIEngine

from kani.exceptions import FunctionCallException


class TrackCallsKani(Kani):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.successful_calls = collections.Counter()
        self.failed_calls = collections.Counter()

    async def handle_function_call_exception(self, call, err, attempt, tool_call_id=None):
        msg = ChatMessage.function(name=call.name, content=str(err), tool_call_id=tool_call_id)
        return ExceptionHandleResult(should_retry=attempt < self.retry_attempts, message=msg)

    async def do_function_call(self, call, *args, **kwargs):
        try:
            res = await super().do_function_call(call, *args, **kwargs)
            self.successful_calls[call.name] += 1
            return res
        except FunctionCallException:
            self.failed_calls[call.name] += 1
            raise

    @ai_function()
    def get_time(self):
        """Get the current time."""
        raise RuntimeError("The time API is offline please try using get_date_and_time")

    @ai_function()
    def get_date_and_time(self):
        """Get the current day and time."""
        return str(datetime.datetime.now())


engine = OpenAIEngine(api_key, model="gpt-4")
ai = TrackCallsKani(engine)
chat_in_terminal(ai)
print(f"Successful Calls: {ai.successful_calls}")
print(f"Failed Calls: {ai.failed_calls}")

USER: What time is it?
AI: Thinking (get_time)...
AI: Thinking (get_date_and_time)...
AI: The current time is 22:26.
Successful Calls: Counter({'get_date_and_time': 1})
Failed Calls: Counter({'get_time': 1})
